Hugo Da Costa, Alban Guerbois
M1 I2D - Projet MachineLearning/DataScience

# Decision Tree et  Random Forest

In [1]:
import numpy as np
import random
%matplotlib inline
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
from sklearn.metrics import confusion_matrix

Soit $X$ la matrice représentant l'ensemble des données que nous avons. 
On note : $ X \in \mathbb{R^{n*d}} $

Chacun de ce des $d-uplets$, soit chaque vecteur de ${R^{d}}$, décrit 1 individu.

# 1. Définition des classes et méthodes de notre structure

In [18]:
class Node(object):
    def __init__(self, father=None):
        self.father = father #type = Node
        self.question = None #type = Question
        self.leftSon = None  #type = Node
        self.rightSon = None  #type = Node
        self.predictedClass = None #type = int ou None (selon si feuille ou pas)
    
    
    def get_depth(self):
        if self.father == None:
            return 0
        return (self.father.get_depth() + 1)

    
# Mutateurs
    def setLeftSon(self, node):
        self.leftSon = node
        
    def setRightSon(self, node):
        self.rightSon = node
        
    def setPrediction(self, label):
        self.predictedClass = label
    
    def setQuestion(self, q):
        self.question = q

Dans toute la suite, on notera subsetOfX et  subsetOfy pour évoquer un certain sous-ensemble des données incarnant le training_set, ou même un sous ensemble de ce training set.

En effet, chaque arbre de décision recevra en entrée un training_set issu des data globales, afin de créer ses noeuds. Il recevra ensuite le test_set afin de déduire leur label, et on vérifiera la qualité de ces prédictions.

In [19]:
def getGiniImpurity(subsetOfy):
    labels, occurences = np.unique(subsetOfy, return_counts=True)
    return 1 - sum( (occurences/np.size(subsetOfy) )**2 )


def getInformationGain(subsetOfy, subOfsub1, subOfsub2):
    gini_init = getGiniImpurity(subsetOfy)
    gini_subset1 = getGiniImpurity(subOfsub1)
    gini_subset2 = getGiniImpurity(subOfsub2)
    
    return gini_init - ( gini_subset1*np.size(subOfsub1)/np.size(subsetOfy) + gini_subset2*np.size(subOfsub2)/np.size(subsetOfy) )
 

In [20]:
class Question(object):
    def __init__(self, testedValue, testedDimension):
        self.testedValue = testedValue
        self.testedDimension = testedDimension
        
    def askQuestion(self, nuplet):
        if nuplet[self.testedDimension] > self.testedValue :
            return True

# 2. Decision Tree

# 2.1 Classe, cas général

On considère que notre set de données se récupère souvent sous la forme d'un DataFrame utilisable via la librairie Pandas.

In [21]:
class DecisionTree(object):
    def __init__(self, root=Node(), max_depth=None):
        self.root = root
        self.max_depth = max_depth
        
    # Dans notre D.T, un noeud est une feuille si sa Gini=0
    def isLeaf(self, subsetOfy):
        if getGiniImpurity(subsetOfy) == 0 :
            return True
        else :
            return False
    
    # ou bien s'il a atteint à la limite de la profondeur_maximale
    def isBeforeMaxDepth(self, node):
        if self.max_depth == None :
            return True
        
        elif self.max_depth > node.get_depth() :
            return True
        
        return False
    
    
    
    # Les fonctions split et best_question sont cruciales : 
    
    # A chaque noeud, on va trouver la question qui apporte le plus d'information sur le jeu de données
    # et on va alors créer 2 fils a ce noeud, accompagnant les 2 sous matrices obtenus en divisant le subset via la question.
    def split(self, current_node, subsetOfX, subsetOfy):
        n, d = subsetOfX.shape
        if (not self.isLeaf(subsetOfy)) and (n>3) and (self.isBeforeMaxDepth(current_node)) :
            X_1, y_1, X_2, y_2 = self.best_question(current_node, subsetOfX, subsetOfy)
            
            # Cas dans lequel la collection contient des éléments identiques sur leurs features,
            # mais dont les labelssont différents.
            if X_1.size == 0 or X_2.size == 0:
                current_node.predictedClass = np.bincount(subsetOfy).argmax()
            
            else :
                self.split(current_node.leftSon, X_1, y_1)
                self.split(current_node.rightSon, X_2, y_2)
        
        else :
            current_node.setPrediction( np.argmax(np.bincount(subsetOfy)) )
        
    
    
    # Cette "meilleure question" elle, s'obtient en testant pleins de questions (sur différentes valeurs et différentes dimensions)
    def best_question(self, node, subsetOfX, subsetOfy):
        n , d = subsetOfX.shape  # n = nb of individuals in the subMatrix of X / d = number of features in...
        
        best_d_test, best_value_test, best_filter, max_gain_info = 0, 0, [], -3
        
        for _ in range(1000):
            
            d_test = random.choice([dim for dim in subsetOfX.columns])
            
            low_bound = min( subsetOfX[d_test] )
            up_bound = max( subsetOfX[d_test] )
            value_test = random.uniform(low_bound, up_bound)
            
            filter1 = subsetOfX[d_test] > value_test
            subsubOfX_1 = subsetOfX[filter1]
            subsubOfy_1 = subsetOfy[filter1]
            
            filter2 = np.logical_not(filter1)
            subsubOfX_2 = subsetOfX[filter2]
            subsubOfy_2 = subsetOfy[filter2]
            
            gain_info = getInformationGain(subsetOfy, subsetOfy[filter1], subsetOfy[filter2])
            
            if max_gain_info < gain_info :
                best_d_test = d_test
                best_value_test = value_test
                best_filter = filter1
                max_gain_info = gain_info
        
        node.setLeftSon( Node(father=node) )
        node.setRightSon( Node(father=node) )
        node.setQuestion( Question(best_value_test, best_d_test) )
        
        return subsetOfX[best_filter], subsetOfy[best_filter], subsetOfX[np.logical_not(filter1)], subsetOfy[np.logical_not(filter1)]
                
    
    
    
    # Tout ce travail sur le Training set et les subsetofTrainingSet était pour obtenir des questions cohérentes
    # nous n'avons plus qu'a les appliquer sur des vecteurs de R^n
    
    def get_tuple_prediction(self, row):
        current = self.root
        while ( (current.leftSon != None)  and (current.rightSon != None) ):
            if row[current.question.testedDimension] > current.question.testedValue :
                current = current.leftSon
            else :
                current = current.rightSon
        return current.predictedClass        
                
    
    
    def test_data(self, X_train, y_train, X_test, y_test):
        # Création des noeuds de l'arbre via le trainig set
        noeud = self.root
        self.split(noeud, X_train, y_train)  #entrainement de notre arbre
        
        # Assignation de tous les classes aux vecteurs tests
        predictions = pd.DataFrame(np.zeros(X_test.shape[0]), index=X_test.index)
        
        for index_row in X_test.index :
            predictions.loc[index_row] = self.get_tuple_prediction(X_test.loc[index_row])

        return predictions, confusion_matrix(y_test, predictions)
      

# 3. Random Forest

In [22]:
class RandomForest(object):
    def __init__(self, nb_trees=10, prop_individuals_byTree=0.25, prop_dimensions_byTree=0.25, max_depth_by_tree=None):
        self.nb_trees=nb_trees
        self.prop1 = prop_individuals_byTree
        self.prop2 = prop_dimensions_byTree
        self.max_depth = max_depth_by_tree
    
    
    
# Chaque Decision Tree de notre Random Forest va recevoir une sous matrice de X_train, sur laquelle il pourra apprendre
# Comme ca, chacun de nos arbres travaillera sur certaines parties du training_set, avec une certaine redondance.

    def get_subpart_of_trainingSet(self, X_train, y_train):
        subX, subY = [], []
        
        number_rows = int(self.prop1 * X_train.shape[0])
        rows_set = set([])
        
        number_dimensions = int(self.prop2 * X_train.shape[1])
        dimensions_set = set([])
        
        while len(rows_set) <= number_rows :
            rand_row = random.choice( [i for i in X_train.index] )
            if not (rand_row in rows_set) :
                rows_set.add(rand_row)
                
        while len(dimensions_set) <= number_dimensions :
            rand_dim = random.choice([d for d in X_train.columns])
            if not (rand_dim in dimensions_set) :
                dimensions_set.add(rand_dim)
        
        #print("individus testés : ", rows_set)
        #print("dimensions testées : ", dimensions_set)
        
        # subX = X_train.iloc[[rows_set, dimensions_set]]
        
        for index_row in rows_set :
            x0 = []
            for index_dim in dimensions_set :
                x0.append(X_train.at[index_row, index_dim])
                
            subX.append(x0)
            subY.append(y_train.loc[index_row])
            
        subX = pd.DataFrame(np.array(subX), index=[i for i in rows_set], columns=[j for j in dimensions_set])
        subY = pd.Series(np.array(subY), index=[i for i in rows_set])
        
        return subX, subY
    
    
    
    
# Redefinition de la prédiction d'un n-uplet = Par la majorité de comment le classe tous les arbres (Vote majoritaire)
 
    def test_data_rf(self, X_train, y_train, X_test, y_test):
        
        predictions_matrix = pd.DataFrame( np.zeros( (self.nb_trees, X_test.shape[0]) ), columns=X_test.index)
        
        compt_trees = 0
        for tree_index in range(self.nb_trees):
            d_tree = DecisionTree(max_depth = self.max_depth)
            subX, subY = self.get_subpart_of_trainingSet(X_train, y_train)
            pred_tree, Mat = d_tree.test_data(subX, subY, X_test, y_test)
            
            compt_trees += 1
            #print("Confusion Matrix of this tree : \n", Mat)
            print(compt_trees, "tree(s) done.")
            #print( "----------------\n\n")
            predictions_matrix.loc[tree_index] = pred_tree[0]
        
        predictions = pd.Series(np.zeros(X_test.shape[0]), index=X_test.index)
        predictions_matrix.loc[:].astype(int)
        
        for data_index in X_test.index  :
            # predictions_matrix = np.array( [ row.astype(int) for row in predictions_matrix.loc[:] ] )
            predictions.loc[data_index] = np.argmax( np.bincount(predictions_matrix[data_index].astype(int)) )
        
        print( "----------------\n\n")
        return predictions, confusion_matrix(y_test, predictions)